# INM707 Coursework
### Aaron Mir (Student Number: 160001207)
<img src="All_Tasks.png" alt="All_TasksOverview" width="700"/>  <img src="Task_1.png" alt="Task_1" width="700"/>

Environment: The agent is preparing for an upcoming mission in which they are to infiltrate an enemy stronghold to gather intelligence on a potential coup d'état. The enemy land is in the form of an NxN grid with each grid containing one stronghold of size N/2 x N or N x N/2 (starting on either side of the middle column (randomly decided)) and a wide river surrounding the entire enemy land. The stronghold has as many entrances from the mainland as (1/4)N and contains dangerous enemy combatants (at random positions) within the stronghold that move around per step (stochasticity). The land inside the stronghold has no transition probability associated with it (deterministic). The land surrounding the stronghold (the shore) is booby-trapped and covered in mist which means that there is both land which can kill/hurt the agent as well as normal land and a transition probability associated with each step (stochasticity) outside the stronghold. The amount of shore that has traps on it is (1/4)N x rows of shore. The row/column of values immediately beside the stronghold is normal land. The goal of the agent is to infiltrate the stronghold and gather the intelligence without being seen/killed by enemy combatants or booby traps.

The agent starts in a random cell which belongs to the land surrounding the stronghold.

Rewards: +10 for getting intelligence, -100 for getting hurt by a combatant or booby trap, -1 for moving into a wall or water

Terminal States: agent moves to the intelligence, if agent moves to a trap and dies, if the agent moves to an enemy combatant.

N represents normal land [0], X represents a river [1], A represents the agent [2], W represents a wall [3], T represents a booby trap [4], E represents an enemy [5], I represents the intelligence [6]


You don't need reward matrix and transition probability for q-learning

Task 1 should be visualisation and defining the stuff


In [66]:
import numpy as np
import matplotlib.pyplot as plt

In [69]:
size = 10
land = np.zeros((size, size))
land[0,:] = 1  
land[:,0] = 1
land[:,size-1] = 1
land[size-1,:] = 1  

In [71]:
column_choice = np.random.choice((size//2-1, size//2+1)) # random choice whether stronghold starts from the left or right of the 'central' column
land[1:size-1, column_choice] = 3
if column_choice == size//2-1:                              # if stronghold is on left
    land[1, 0:column_choice] = 3                            # establish the walls of stronghold
    land[size-2, 0:column_choice] = 3
    land[1:size-1, 0] = 3                                   
    for col in land[1:size-1, column_choice+2:size-1].T:
        traps = []
        for i in range(int(np.round(1/4*len(col)))):        # make as many traps as 1/4 of the length of each column in the shore
            trap = np.random.choice(np.setdiff1d(range(len(col)), traps))
            col[trap] = 4
            traps.append(trap)
    for col in land[2:size-2, 1:column_choice].T:
        enemies = []
        for i in range(int(np.round(1/4*len(col)))):        # populate enemies randomly inside the stronhold
            enemy = np.random.choice(np.setdiff1d(range(len(col)), enemies))
            col[enemy] = 5
            enemies.append(enemy)
    intel_row = np.random.randint(1, len(land[2:size-2, 1:column_choice-1]))
    intel_col = np.random.randint(1, len(land[2:size-2, 1:column_choice-1].T))
    land[intel_row+2][column_choice-intel_col] = 6            # randomly insert intelligence into stronghold 

if column_choice == size//2+1:                              # if stronghold is on right
    land[1, column_choice:size] = 3
    land[size-2, column_choice:size] = 3
    land[1:size-1, size-1] = 3
    for col in land[1:size-1, 1:column_choice-1].T:
        traps = []
        for i in range(int(np.round(1/2*len(col)))):        # make as many traps as 1/4 of the length of each column in the shore
            trap = np.random.choice(np.setdiff1d(range(len(col)), traps))
            col[trap] = 4
            traps.append(trap)
    for col in land[2:size-2, column_choice+1:size-1].T:
        enemies = []
        for i in range(int(np.round(1/4*len(col)))):        # populate enemies randomly inside the stronhold
            enemy = np.random.choice(np.setdiff1d(range(len(col)), enemies))
            col[enemy] = 5
            enemies.append(enemy)
    intel_row = np.random.randint(1, len(land[2:size-2, column_choice+1:size-1]))              
    intel_col = np.random.randint(1, len(land[2:size-2, column_choice+1:size-1].T))
    land[intel_row+2][column_choice+intel_col] = 6            # randomly insert intelligence into stronghold

entrances = []
for i in range(int(np.round(1/4*len(land[2:size-2, column_choice])))+1):        # make as many entrances as 1/4 of the length of the front wall
    entrance = np.random.choice(np.setdiff1d(range(len(land[2:size-1, column_choice])), entrances))
    land[2:size-1, column_choice][entrance] = 0
    entrances.append(entrance)
print(land)    

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [3. 3. 3. 3. 3. 0. 3. 3. 3. 3.]
 [3. 5. 0. 4. 0. 0. 3. 0. 5. 3.]
 [3. 4. 5. 5. 0. 0. 4. 0. 0. 3.]
 [3. 4. 0. 5. 0. 0. 3. 4. 0. 3.]
 [3. 5. 5. 4. 3. 0. 3. 5. 4. 3.]
 [3. 4. 4. 0. 3. 0. 0. 4. 0. 3.]
 [3. 0. 0. 6. 3. 0. 4. 5. 4. 3.]
 [3. 3. 3. 3. 3. 0. 0. 3. 3. 3.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [72]:
print(intel_row, intel_col)

5 1


In [65]:
np.random.randint(1, len(land[2:size-2, column_choice+1:size-1]))

1

In [ ]:
class Stronghold():
    def __init__(self, size):
        self.size = size
        self.actions = {0: 'Up', 1: 'Down', 2: 'Left', 3: 'Right'}
        self.land = self.env_gen()

    def env_gen(self):
        land = np.zeros((self.size, self.size))
        land[0,:] = 1
        land[:,0] = 1
        land[self.size-1,:] = 1
        land[:,self.size-1] = 1
        return land
    
    def step(self, action):
        
        return next_state, reward, done
    
    def reset():
        pass

    def render(): # displays
        pass
''' Don't need this for Q-learning
    def fill_transition_probability(self): # p.106
        
        return P
    
    def fill_reward_matrix(self):
        
        return R
'''

In [ ]:
class Policies():
    def __init__(self):
        pass
    
    def random_policy(self):
        pass

In [ ]:
env = Stronghold(10)

In [ ]:
    # if row_choice == size//2:
    #     choice_side = np.random.choice(('up', 'down'))
    #     print(choice_side)
    #     if choice_side == 'up':
    #         row_choice = size//2-1
    #     if choice_side == 'down':
    #         row_choice = size//2+1

    # if column_choice == size//2:
    #     choice_side = np.random.choice((size//2-1, size//2+1)) # random choice between left and right
    #     print(choice_side)
    #     column_choice = choice_side

In [ ]:
# choice = np.random.choice(('row', 'column')) # randomly deciding whether the stronghold will start 1 cell away from the middle row or middle column.
# if choice == 'row':
#     row_choice = np.random.choice((size//2-1, size//2+1))       # random choice between up or down cell
#     land[row_choice, 1:size-1] = 3
#     if row_choice == size//2-1:
#         land[0:row_choice, 1] = 3
#         land[0:row_choice, size-2] = 3
#         land[0, 1:size-1] = 3
#         land[1:row_choice]
#         for row in land[row_choice+2:size-1, 1:size-1]:
#             traps = []
#             for i in range(int(np.round(1/4*len(row)))):        # make as many traps as 1/4 of the length of each row in the shore
#                 trap = np.random.choice(np.setdiff1d(range(len(row)), traps))
#                 row[trap] = 4
#                 traps.append(trap)
#     if row_choice == size//2+1:
#         land[row_choice:size, 1] = 3
#         land[row_choice:size, size-2] = 3
#         land[size-1, 1:size-1] = 3
#         for row in land[1:row_choice-1, 1:size-1]:
#             traps = []
#             for i in range(int(np.round(1/2*len(row)))):        # make as many traps as 1/4 of the length of each row in the shore
#                 trap = np.random.choice(np.setdiff1d(range(len(row)), traps))
#                 row[trap] = 4
#                 traps.append(trap)
#     entrances = []
#     for i in range(int(np.round(1/4*len(land[row_choice,2:size-2])))+1):        # make as many entrances as 1/4 of the length of the front wall
#         entrance = np.random.choice(np.setdiff1d(range(len(land[row_choice,2:size-1])), entrances))
#         land[row_choice,2:size-1][entrance] = 0
#         entrances.append(entrance)